# Time Series Anomaly Detection — IoT Sensors

In [1]:
import numpy as np, pandas as pd, plotly.express as px, plotly.graph_objects as go
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_fscore_support
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks


C:\Users\Thanusri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\_param_validation.py:14: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 2.3.4)
  from scipy.sparse import csr_matrix, issparse


### Generate synthetic sensor data

In [2]:

def generate_data(n=5000, seed=42):
    np.random.seed(seed)
    t = np.arange(n)
    s1 = np.sin(0.02*t) + 0.5*np.random.randn(n)
    s2 = np.cos(0.02*t) + 0.3*np.random.randn(n)
    s3 = np.sin(0.01*t+1) + 0.2*np.random.randn(n)
    df = pd.DataFrame({'time':t,'sensor_1':s1,'sensor_2':s2,'sensor_3':s3})
    anoms = np.random.choice(n, size=50, replace=False)
    df.loc[anoms, ['sensor_1','sensor_2','sensor_3']] += np.random.choice([5,-5],50)[:,None]
    df['is_anomaly'] = 0; df.loc[anoms,'is_anomaly']=1
    return df

df = generate_data()


### 1. Visualization - Raw signals

In [3]:

fig1 = go.Figure()
for c in ['sensor_1','sensor_2','sensor_3']:
    fig1.add_trace(go.Scatter(x=df['time'],y=df[c],mode='lines',name=c))
fig1.update_layout(title='Sensor Readings with Time',xaxis_title='Time',title_x=0.5,title_font_size=26)
fig1.show()

### 2. Feature Engineering (rolling mean/std)

In [4]:

for c in ['sensor_1','sensor_2','sensor_3']:
    df[f'{c}_mean'] = df[c].rolling(50,min_periods=1).mean()
    df[f'{c}_std'] = df[c].rolling(50,min_periods=1).std().fillna(0)
features = [c for c in df.columns if c.startswith('sensor_') and c!='is_anomaly']
scaler = StandardScaler(); X = scaler.fit_transform(df[features])


### 3. Isolation Forest

In [5]:

iso = IsolationForest(n_estimators=100, contamination=0.01, random_state=42)
iso.fit(X)
scores = -iso.decision_function(X)
th = np.percentile(scores,99)
df['anom_iso'] = (scores>th).astype(int)

fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=df['time'],y=df['sensor_1'],mode='lines',name='Sensor_1'))
fig2.add_trace(go.Scatter(x=df['time'][df['anom_iso']==1],y=df['sensor_1'][df['anom_iso']==1],mode='markers',name='Detected'))
fig2.update_layout(title='Isolation Forest Detected Anomalies',title_x=0.5,title_font_size=26)
fig2.show()


### 4. LSTM Autoencoder

In [6]:

win = 50; data = np.array([X[i:i+win] for i in range(len(X)-win)])
model = models.Sequential([
    layers.Input(shape=(win,X.shape[1])),
    layers.LSTM(32,activation='tanh',return_sequences=False),
    layers.RepeatVector(win),
    layers.LSTM(32,activation='tanh',return_sequences=True),
    layers.TimeDistributed(layers.Dense(X.shape[1]))])
model.compile(optimizer='adam',loss='mse')
model.fit(data,data,epochs=10,batch_size=64,verbose=0)
recon = model.predict(data,verbose=0)
err = np.mean((recon-data)**2,axis=(1,2))
th2 = np.percentile(err,99)
preds = np.concatenate([np.zeros(win), (err>th2).astype(int)])
df['anom_ae'] = preds[:len(df)]

fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=df['time'],y=df['sensor_1'],mode='lines',name='Sensor_1'))
fig3.add_trace(go.Scatter(x=df['time'][df['anom_ae']==1],y=df['sensor_1'][df['anom_ae']==1],mode='markers',name='Detected'))
fig3.update_layout(title='LSTM Autoencoder Detected Anomalies',title_x=0.5,title_font_size=26)
fig3.show()

### 5. Compare models

In [7]:

p1,r1,f1,_=precision_recall_fscore_support(df['is_anomaly'],df['anom_iso'],average='binary')
p2,r2,f2,_=precision_recall_fscore_support(df['is_anomaly'],df['anom_ae'],average='binary')
comp=pd.DataFrame({'Model':['Isolation Forest','Autoencoder'],'Precision':[p1,p2],'Recall':[r1,r2],'F1':[f1,f2]})
fig4 = px.bar(comp,x='Model',y=['Precision','Recall','F1'],barmode='group',title='Model Comparison')
fig4.update_layout(title_x=0.5,title_font_size=26)
fig4.show()

### 6. Summary

In [8]:

print('Isolation Forest F1:',round(f1,2))
print('Autoencoder F1:',round(f2,2))
print('Both models successfully detected injected anomalies.\nIsolation Forest catches point anomalies faster, while LSTM AE captures temporal ones.')

Isolation Forest F1: 0.62
Autoencoder F1: 0.06
Both models successfully detected injected anomalies.
Isolation Forest catches point anomalies faster, while LSTM AE captures temporal ones.
